In [1]:
import sys
sys.path.insert(0,'/home/mohit.kumargupta/deep_boltzmann')
import numpy as np
import mdtraj as md

In [3]:
TrajDirectory = "/home/mohit.kumargupta/confALA4/"
TrajFile = TrajDirectory +  "traj_comp.xtc"
TopFile = TrajDirectory + "ala4_amber_atA.gro"
def alnTetra(TrajFile, Topfile,i):
    xtObject = md.load(TrajFile, top = Topfile)
    return xtObject.xyz[i*500:(i+1)*500]


In [2]:
import mdtraj as md
import sys

In [ ]:
%load_ext memory_profiler
mdObj = alnTetra(TrajFile, TopFile,0)
trainingConfigs = mdObj[:1000].flatten().reshape((1000,10557))
device = "cuda"

In [ ]:
print(trainingConfigs.shape)

In [ ]:
print(trainingConfigs[0].max())

In [4]:
sz =16

In [5]:
boxes =[]
x=0
while(x<3.3):
    y=0.0
    while(y<3.3):
        z=0.0
        while(z<3.31):
            boxes.append([x+ 3.4/(2*sz),y+3.4/(2*sz),z+3.4/(2*sz)])
            print(z)
            z+=3.4/sz
            
        y+= 3.4/sz
    x+= 3.4/sz

0.0
0.2125
0.425
0.6375
0.85
1.0625
1.275
1.4874999999999998
1.6999999999999997
1.9124999999999996
2.1249999999999996
2.3374999999999995
2.5499999999999994
2.7624999999999993
2.974999999999999
3.187499999999999
0.0
0.2125
0.425
0.6375
0.85
1.0625
1.275
1.4874999999999998
1.6999999999999997
1.9124999999999996
2.1249999999999996
2.3374999999999995
2.5499999999999994
2.7624999999999993
2.974999999999999
3.187499999999999
0.0
0.2125
0.425
0.6375
0.85
1.0625
1.275
1.4874999999999998
1.6999999999999997
1.9124999999999996
2.1249999999999996
2.3374999999999995
2.5499999999999994
2.7624999999999993
2.974999999999999
3.187499999999999
0.0
0.2125
0.425
0.6375
0.85
1.0625
1.275
1.4874999999999998
1.6999999999999997
1.9124999999999996
2.1249999999999996
2.3374999999999995
2.5499999999999994
2.7624999999999993
2.974999999999999
3.187499999999999
0.0
0.2125
0.425
0.6375
0.85
1.0625
1.275
1.4874999999999998
1.6999999999999997
1.9124999999999996
2.1249999999999996
2.3374999999999995
2.5499999999999994


2.7624999999999993
2.974999999999999
3.187499999999999
0.0
0.2125
0.425
0.6375
0.85
1.0625
1.275
1.4874999999999998
1.6999999999999997
1.9124999999999996
2.1249999999999996
2.3374999999999995
2.5499999999999994
2.7624999999999993
2.974999999999999
3.187499999999999
0.0
0.2125
0.425
0.6375
0.85
1.0625
1.275
1.4874999999999998
1.6999999999999997
1.9124999999999996
2.1249999999999996
2.3374999999999995
2.5499999999999994
2.7624999999999993
2.974999999999999
3.187499999999999
0.0
0.2125
0.425
0.6375
0.85
1.0625
1.275
1.4874999999999998
1.6999999999999997
1.9124999999999996
2.1249999999999996
2.3374999999999995
2.5499999999999994
2.7624999999999993
2.974999999999999
3.187499999999999
0.0
0.2125
0.425
0.6375
0.85
1.0625
1.275
1.4874999999999998
1.6999999999999997
1.9124999999999996
2.1249999999999996
2.3374999999999995
2.5499999999999994
2.7624999999999993
2.974999999999999
3.187499999999999
0.0
0.2125
0.425
0.6375
0.85
1.0625
1.275
1.4874999999999998
1.6999999999999997
1.9124999999999996
2.

In [6]:
boxes = np.array(boxes)

In [ ]:
boxes = np.tile(boxes,(3519-42,1,1))

In [ ]:
boxes = np.tile(boxes, (5,1,1,1))

In [ ]:
boxes.shape

In [7]:
vol = (3.4/sz) **3
box_size= 3.4

In [ ]:
dataSet32 = []
i = 0

In [ ]:
dataSet32 = None

In [ ]:
from multiprocessing import Pool,  Process, Lock

In [ ]:
def Dmap(trainingConfigs):
    global boxes
    dataSet32 = None
    for i in range(0,int(trainingConfigs.shape[0]/boxes.shape[0]) ):
        i+=1
        print(i)
        snap = np.transpose( np.tile(np.reshape(trainingConfigs[i*5:(i+1)*5,42*3:],(5,3477,3) ), (256*16,1,1,1)), (1,2,0,3))
        print(snap.shape)
        dist = np.linalg.norm(snap-boxes,axis = 3)
        val =  vol * np.exp(-0.5*np.square(dist/0.5))/ ( ( (2*np.pi)**(1.5) ) * 0.125)
        snap = None
        if(dataSet32 is None):
            dataSet32= val.sum(axis=1)
        else:
            dataSet32 = np.vstack([dataSet32,val.sum(axis=1)] )
    return dataSet32

In [ ]:
%%time
for i in range(0,int(trainingConfigs.shape[0]/boxes.shape[0]) ):
    i+=1
    print(i)
    snap = np.transpose( np.tile(np.reshape(trainingConfigs[i*10:(i+1)*10,42*3:],(10,3477,3) ), (256*16,1,1,1)), (1,2,0,3))
    print(snap.shape)
    dist = np.linalg.norm(snap-boxes,axis = 3)
    val =  vol * np.exp(-0.5*np.square(dist/0.5))/ ( ( (2*np.pi)**(1.5) ) * 0.125)
    snap = None
    if(dataSet32 is None):
        dataSet32= val.sum(axis=1)
    else:
        dataSet32 = np.vstack([dataSet32,val.sum(axis=1)] )

In [ ]:
for j in range(60,100):
    mdObj = xtObject.xyz[j*500:(j+1)*500]
    trainingConfigs = mdObj[:500].flatten().reshape((500,10557))
    dataSet32 = []
    oxygen = []
    hydrogen = []
    print(j)
    i=0
    for Snapshot in trainingConfigs:
        print(i)
        i+=1
        snap = np.transpose( np.tile(np.reshape(Snapshot[42*3:],(3477,3) ), (256*16,1,1)), (1,0,2))
        dx = np.absolute(snap-boxes)
        print(dx.shape)
        dist = np.linalg.norm(dx-box_size*np.floor( 0.5 +  dx/box_size),axis = 2)
        #dist = np.linalg.norm(dx,axis=2)
        #print(dist.shape)
        val =  vol * np.exp(-0.5*np.square(dist/0.1))/ ( (2*np.pi * 0.01)**1.5)
        oxygen.append(val[::3].sum(axis=0))
        hydrogen.append( (val[1::3]+ val[2::3]).sum(axis=0))

        #print(val.sum(axis=0).shape)
    print(np.vstack([np.expand_dims(np.array(oxygen), 0), np.expand_dims(np.array(hydrogen),0)] ))
    #np.save('ProteinData/ProteinData' + str(j), np.array(dataSet32))

In [8]:
xtObject = md.load(TrajFile, top = TopFile)

In [9]:
import torch
import torch.nn as nn
from pathlib import Path
import torchvision.transforms as transforms
import torch.optim as optim
import numpy as np
from math import *
from random import gauss,seed
import torch.nn.functional as F
import math
import matplotlib.pyplot as plt
import seaborn

In [10]:
class DensityEstimator(nn.Module):
    def __init__(self,dimer_atoms,output_dim):
        super(DensityEstimator, self).__init__()
        self.dimer_atoms = dimer_atoms
        self.hidden_dim = 256
        self.relu = nn.LeakyReLU(0.2, inplace=True)
        self.fc1 = nn.Linear(self.dimer_atoms, self.hidden_dim)
        self.fc2 = nn.Linear(self.hidden_dim, int(self.hidden_dim/2) )
        self.fc3 = nn.Linear(int(self.hidden_dim/2), int(self.hidden_dim/4))

    def forward(self,x):
        out = self.relu( self.fc1(x) )
        out = self.relu(self.fc2(out))
        out = self.fc3(out)
        out = 1160*torch.softmax(out,1)
        return out



In [11]:
class PixShuf(nn.Module):
    def __init__(self, up):
        super(PixShuf,self).__init__()
        self.upsample = up
    def forward(self,inp):
        out = inp.reshape((inp.shape[0],inp.shape[1]//(self.upsample**3), self.upsample,self.upsample, self.upsample, inp.shape[2],inp.shape[3],inp.shape[4]))
        out = out.permute(0,1,5,2,6,3,7,4)
        out = out.reshape((out.shape[0],out.shape[1],inp.shape[2]*self.upsample,inp.shape[3]*self.upsample,inp.shape[4]*self.upsample))
        return out
        

In [12]:
class RESNET(nn.Module):
    def __init__(self,numberOfLayers= 18, dim = 4):
        super(RESNET, self).__init__()
        self.conv_input = nn.Conv3d(in_channels=1, out_channels=64, kernel_size=5, stride=1, padding=2, bias=False)
        self.relu = nn.LeakyReLU(0.2, inplace=True)
        self.residual = self.make_layer(Residual_Block, numberOfLayers)
        self.upscale2x = nn.Sequential(
            nn.Conv3d(in_channels=64, out_channels=256*2, kernel_size=3, stride=1, padding=1, bias=False),
            PixShuf(2),
            nn.LeakyReLU(0.2, inplace=True),
        )
        self.conv_output = nn.Conv3d(in_channels=64, out_channels=1, kernel_size=5, stride=1, padding=2, bias= False)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()

    def make_layer(self, block, num_of_layer):
        layers = []
        for _ in range(num_of_layer):
            layers.append(block())
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.relu(self.conv_input(x))
        residual = out
        out = self.residual(out)
        out = torch.add(out,residual)
        #self.relu(F.pixel_shuufle)
        #out = self.relu( F.pixel_shuffle( self.UpScaleConv(out), 2) )
        out = self.upscale2x(out)
        out = self.conv_output(out)
        return out
       

In [13]:
class Residual_Block(nn.Module):
    def __init__(self):
        super(Residual_Block, self).__init__()

        self.conv1 = nn.Conv3d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1, bias=False)
        self.in1 = nn.InstanceNorm3d(64, affine=True)
        self.relu = nn.LeakyReLU(0.2, inplace=True)
        self.conv2 = nn.Conv3d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1, bias=False)
        self.in2 = nn.InstanceNorm3d(64, affine=True)

    def forward(self, x):
        identity_data = x
        output = self.in1( self.relu(self.conv1(x)) )
        output = self.in2( self.conv2(output) )
        output = torch.add(output,identity_data)
        return output

In [14]:
class CombineModel(nn.Module):
    def __init__(self,model1,model2,model3):
        super(CombineModel, self).__init__()
        self.model1 = model1
        self.model2 = model2
        self.model3 = model3
    
    def forward(self,inp):
        out = self.model1(inp)
        out = torch.reshape(out,(inp.shape[0],1,4,4,4) )
        print(out.shape)
        out = self.model2(out)
        out = self.model3(out)
        return out

In [15]:
class ConfigurationModel(nn.Module):
    
    def __init__(self,model1):
        super(ConfigurationModel,self).__init__()
        self.model1 = model1
        self.hidden_dim = 16*16
        self.out_dim = 11
        self.relu = nn.LeakyReLU(0.2, inplace=True)
        self.fc1 = nn.Linear(256*16, 100)
        self.fc2 = nn.Linear(100, 1164*3)
    
    def forward(self,inp):
        out = self.model1(inp)
        out = out.reshape((inp.shape[0],256*16))
        out = self.relu(self.fc1(out))
        out = self.relu(self.fc2(out))
        print(out.shape)
        return out

In [ ]:
a = np.load('ProteinData/ProteinData' + str(188)+ '.npy')

In [ ]:
m1= DensityEstimator(42*3,64).to('cuda')

In [ ]:
m2 = RESNET(9,dim=4).to('cuda')

In [ ]:
m3 = RESNET(9,dim=8).to('cuda')

In [ ]:
comModel = CombineModel(m1,m2,m3).to('cuda')

In [16]:
perm = np.arange(160)

In [17]:
configModel = ConfigurationModel(comModel).to('cuda')

NameError: name 'comModel' is not defined

In [18]:
validPerm = np.arange(20)+160

In [19]:
print(validPerm)

[160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177
 178 179]


In [20]:
adOptimizer = optim.Adam(configModel.parameters(),lr =0.0001)

NameError: name 'configModel' is not defined

In [21]:
def findNeighbourList():
    global boxes
    bx = []
    for p,eachBox in enumerate(boxes):
        box = []
        for i in range(5) :
            for j in range(5):
                for k in range(5):
                    x= (int(p/256) + i-2)%16
                    y=  ( int( (p%256)/16 ) +j-2)%16
                    z = ( int(p%16) + k-2 ) %16
                    while(x<0):
                        x+=16
                    while(y<0):
                        y+=16
                    while(z<0):
                        z+=16
                    box.append(np.array(boxes[x*256 +y*16 +z] ) )
        bx.append(np.array(box))
    return bx

In [22]:
Epochs = 100

In [23]:
NeighbourList = np.array(findNeighbourList())

In [ ]:
xtObject = md.load(TrajFile, top = TopFile)

In [24]:
NeighbourList = torch.from_numpy(NeighbourList).to('cuda').type('torch.FloatTensor')

In [25]:
NeighbourList.shape

torch.Size([4096, 125, 3])

In [26]:
box_size= 3.4
sz =16

In [27]:
temp = xtObject.xyz[:200]

In [28]:
def DensityMapCalc(traj):
    DensityMap = None
    box_size =3.4
    for i,snapshot in enumerate(traj):
        print(i)
        print(snapshot.type())
        print(snapshot.shape)
        snapshot= snapshot.to('cuda')
        myBox = (torch.floor((snapshot+box_size)*sz/box_size)%16 ).type('torch.IntTensor')
        myBox = myBox[:,0]*256 + myBox[:,1]*16 + myBox[:,2]

        MyList = []
        for box in myBox:
            MyList.append(NeighbourList[box].detach().to('cpu').numpy())

        MyList = torch.from_numpy(np.array(MyList)).type('torch.FloatTensor')
        MyList = MyList.to('cuda')
        
        dx= torch.abs( (snapshot.unsqueeze(1).expand(-1,125,-1) -MyList) )
        dist = torch.norm( dx - box_size*torch.floor(0.5+dx/box_size),dim=2)
        prob = torch.exp(-0.5 * torch.mul(dist/0.2,dist/0.2))/((2*3.14*0.04)**1.5 )

        MyList = torch.floor( (MyList*sz+ 0.01)/box_size )
        MyList = (MyList[:,:,0]*256 + MyList[:,:,1]*16  + MyList[:,:,2]).type('torch.IntTensor')

        newList = []
        for j in range(MyList.shape[0]):
            for k in range(125):
                newList.append(j)
        newList = torch.from_numpy(np.array(newList))
        #print(MyList[1,0],MyList.flatten()[125])
        out = torch.stack((newList.type('torch.IntTensor'),MyList.flatten()))
        dMap = torch.sparse.FloatTensor(out.type('torch.cuda.LongTensor'),prob.flatten())
        dMap = torch.sparse.sum(dMap,0)*vol
        if(DensityMap is None):
            DensityMap = dMap.unsqueeze(0)
        else:
            DensityMap = torch.cat((DensityMap,dMap.unsqueeze(0)))
    return DensityMap

In [ ]:
%%time
print(DensityMapCalc(temp).shape)

In [ ]:
def DensitylossFunction(out,sample):
    Density = DensityMapCalc(out)
    return MSELoss()(out,sample)

<font color='red'>To Look at Lost Function </font>

In [42]:

%%time
for i,snapshot in enumerate(temp):
    print(i)
    snapshot= torch.from_numpy(snapshot).to('cuda')
    myBox = (torch.floor((snapshot+box_size)*sz/box_size)%16 ).type('torch.IntTensor')
    myBox = myBox[:,0]*256 + myBox[:,1]*16 + myBox[:,2] #Finding Box Number
   
    MyList = []
    for box in myBox:
        MyList.append(NeighbourList[box].detach().to('cpu').numpy())
       
    MyList = torch.from_numpy(np.array(MyList)).type('torch.FloatTensor')
    MyList = MyList.to('cuda')
    
    box_size =3.4
    dx= torch.abs( (snapshot.unsqueeze(1).expand(-1,125,-1) -MyList) )
    dist = torch.norm( dx - box_size*torch.floor(0.5+dx/box_size),dim=2) #calculating distance using minimum image
    prob = torch.exp(-0.5 * torch.mul(dist/0.2,dist/0.2))/((2*3.14*0.04)**1.5 )
    
    MyList = torch.floor( (MyList*sz+ 0.01)/box_size )
    MyList = (MyList[:,:,0]*256 + MyList[:,:,1]*16  + MyList[:,:,2]).type('torch.IntTensor')
    print(MyList.shape,MyList[0,0])
    
    dMap = torch.ones(4096) - 1.0
    break
    for j in range(4096):
            idx = (MyList==j).nonzero()
    
    '''
    newList = []
    for j in range(MyList.shape[0]):
        for k in range(125):
            newList.append(j)
    newList = torch.from_numpy(np.array(newList))
    print(MyList[1,0],MyList.flatten()[125])
    out = torch.stack((newList.type('torch.IntTensor'),MyList.flatten()))
    dMap = torch.sparse.FloatTensor(out.type('torch.cuda.LongTensor'),prob.flatten())
    dMap = torch.sparse.sum(dMap,0)*vol 
    '''
    '''
    break
    newList = None
    for (j,atom) in enumerate(MyList):
        if(newList is None):
            newList = torch.stack( (torch.ones(125,dtype = torch.int).new_full((1,125),j).squeeze(0), MyList[j]) ).permute(1,0)
        else:
            newList= torch.cat( (torch.stack( (torch.ones(1,dtype = torch.int).new_full((1,125),j).squeeze(0) , MyList[j]),0 ).permute(1,0), newList ) )
    #newList = torch.from_numpy(np.array(newList)).to('cuda').type('torch.cuda.LongTensor')
    print(newList.t().shape)
    d = torch.sparse.FloatTensor(newList.t().type('torch.cuda.LongTensor'),prob.flatten())
    #print(torch.sparse.sum(d,0)*vol)
    break
    '''

0
torch.Size([3519, 125]) tensor(808, dtype=torch.int32)
CPU times: user 36.7 ms, sys: 0 ns, total: 36.7 ms
Wall time: 35.5 ms


In [ ]:
i = torch.LongTensor([[0, 2], [1, 0], [1, 2]])
print(i.shape)

In [ ]:
print(xtObject.xyz.min())

In [ ]:
prev= None

In [ ]:
ValidationLoss = []

In [ ]:
for epoch in range(Epochs):
    print(epoch)
    np.random.shuffle(perm)
    i = 0
    totalLoss = 0.0
    for eachVal in perm:
        if(eachVal == 31):
            continue
        print(i,eachVal)
        adOptimizer.zero_grad()
        sample = torch.from_numpy(np.load('ProteinData/ProteinData' + str(eachVal)+ '.npy')[0] ).type('torch.FloatTensor').to('cuda')
        mdObj = torch.from_numpy(xtObject.xyz[eachVal*500:(eachVal+1)*500,:42].reshape(500,42*3)).type('torch.FloatTensor')
        print(sample.sum()/500)
        out = configModel(mdObj.to('cuda'))
        print(out.shape)
        loss = DensitylossFunction(out,sample)
        '''
        loss = (torch.abs(out-sample)/sample).mean()
        mdObj = None
        sample = None
        '''
        print(loss.item())
        totalLoss += loss.item()
        i+=1
        loss.backward()
        adOptimizer.step()
    print(totalLoss)
    
    

In [ ]:
for epoch in range(Epochs):
    print(epoch)
    np.random.shuffle(perm)
    i = 0
    totalLoss = 0.0
    for eachVal in perm:
        if(eachVal == 31):
            continue
        print(i,eachVal)
        adOptimizer.zero_grad()
        sample = torch.from_numpy(np.load('ProteinData/ProteinData' + str(eachVal)+ '.npy')[0] ).type('torch.FloatTensor').to('cuda')
        print(sample.shape)
        mdObj = torch.from_numpy(xtObject.xyz[eachVal*500:(eachVal+1)*500,:42].reshape(500,42*3)).type('torch.FloatTensor')
        print(sample.sum()/500)
        out = comModel(mdObj.to('cuda'))
        '''
        if(prev is None):
            prev = out
        else:
            print(out-prev)
            prev = out
        '''
        #print(out)
        #print(out)
        sample = sample.reshape((500,1,16,16,16))
        print( (torch.abs(out-sample)/sample).mean())
        #print(out.shape,sample.reshape((500,16,16,16)))
        #loss =  torch.sqrt(nn.MSELoss()(out,sample) )
        loss = (torch.abs(out-sample)/sample).mean()
        mdObj = None
        sample = None
        print(loss.item())
        totalLoss += loss.item()
        i+=1
        loss.backward()
        adOptimizer.step()
    print(totalLoss)
    '''
    validLoss =0.0
    np.random.shuffle(validPerm)
    for eachVal in validPerm:
        print(i,eachVal)
        sample = torch.from_numpy(np.load('ProteinData/ProteinData' + str(eachVal)+ '.npy')[0] ).type('torch.FloatTensor').to('cuda')
        mdObj = torch.from_numpy(xtObject.xyz[eachVal*500:(eachVal+1)*500,:42].reshape(500,42*3)).type('torch.FloatTensor')
        out = comModel(mdObj.to('cuda'))
        sample = sample.reshape((500,1,16,16,16))
        print( (torch.abs(out-sample)/sample).mean())
        #print(out.shape,sample.reshape((500,16,16,16)))
        #loss =  torch.sqrt(nn.MSELoss()(out,sample) )
        loss = (torch.abs(out-sample)/sample).mean()
        print(loss.item())
        validLoss += loss.item()
        i+=1
    ValidationLoss.append(validLoss)
    '''

In [ ]:
sample = torch.from_numpy(np.load('ProteinData/ProteinData' + str(120)+ '.npy') ).unsqueeze(1).type('torch.FloatTensor').to('cuda')


In [ ]:
mdObj = torch.from_numpy(alnTetra(TrajFile, TopFile,120)[:500,:42].reshape(500,42*3)).type('torch.FloatTensor')
print(mdObj.shape)